In [118]:
import os  # Standard Python module to interact with the operating system (e.g., env variables, file paths)

from openai import OpenAI  # Official OpenAI client – used for making API calls to GPT-4, GPT-3.5, etc.

from langgraph.graph import StateGraph, END  
# LangGraph is used to define stateful graphs (multi-agent or workflow pipelines)
# StateGraph defines nodes and transitions, END is used to mark terminal states

from typing import TypedDict, Annotated  , Literal
# Built-in Python module for type hints
# TypedDict helps define structured dictionary types
# Annotated lets you attach metadata to types (useful in LangChain input/output schemas)

import operator  
# Built-in Python module providing functional equivalents of operators (like <, ==, etc.)

from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# These are message types in LangChain's core message passing system
# Used for defining and processing chat-style inputs/outputs

from langchain_openai import ChatOpenAI  
# LangChain wrapper for OpenAI's chat models – lets you call GPT-4/3.5 with extra features like streaming

from langchain_community.tools.tavily_search import TavilySearchResults  
# Community-contributed LangChain integration for Tavily (a web search tool for real-time search results)

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage


In [119]:
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("GITHUB_TOKEN")
tavily_api_key = os.getenv("TAVILY_API_KEY")
endpoint = "https://models.github.ai/inference"
model_name = "microsoft/Phi-4"

In [120]:
# In previous examples we've annotated the `messages` state key
# with the default `operator.add` or `+` reducer, which always
# appends new messages to the end of the existing messages array.

# Now, to support replacing existing messages, we annotate the
# `messages` key with a customer reducer function, which replaces
# messages with the same `id`, and appends them otherwise.

In [121]:
from uuid import uuid4
def merge_messages(old: list[AnyMessage], new: list[AnyMessage]) -> list[AnyMessage]:
    for msg in new:
        if not msg.id:
            msg.id = str(uuid4()) # Assign unique ID if missing
    return old + new  # Combine old and new messages


In [122]:
class AgesntState(TypedDict):
    messages: Annotated[list[AnyMessage], merge_messages]
    approved: bool
    retry_count: int
    rejections: list[str]

In [123]:
model = ChatOpenAI(
    model="microsoft/Phi-4",
    base_url="https://models.github.ai/inference",
    api_key=token,
    streaming = True,
)

In [124]:
# --------------------------------------
# ✅ Node 1: Call the LLM and generate a response
# --------------------------------------
def suggest_answer(state: AgentState):
     # Extract message history
    messages = state["messages"]

    # Call the model to generate a reply
    response = model.invoke(messages)

    # Return new message to be merged into state
    return {"messages": [response]}

In [125]:
# --------------------------------------
# ✅ Node 2: Human-in-the-loop approval step
# --------------------------------------
def human_approval(state: AgentState):
    # Extract latest AI response from msg list
    latest_ai_msg = state["messages"][-1].content

    # Display it to human
    print("\n LLM Suggested:")
    print(latest_ai_msg)

    #Ask for approval from the human
    decision = input("Approve this response? (y/n): ").strip().lower()

    approved = decision == "y"

    return {"Approved": approved}

In [132]:
# --------------------------------------
# ✅ Define the LangGraph workflow
# --------------------------------------
graph = StateGraph(AgentState)
# Add the LLM node
graph.add_node("llm", suggest_answer)

# Add the human approval node
graph.add_node("hitl", human_approval)

# Set the entry point of the graph to be the LLM
graph.set_entry_point("llm")

# After LLM responds, go to the HITL node
graph.add_edge("llm", "hitl")

# After human approval:
# - If approved, go to END
# - If rejected, go back to the LLM to retry
graph.add_conditional_edges(
    "hitl",                        # Node to check condition from
    lambda state: state["approved"],  # Condition function (True/False)
    {
        True: END,                # End graph if approved
        False: "llm",             # Retry LLM if rejected
    }
)

# Compile the graph into a runnable workflow
workflow = graph.compile()

In [133]:
# --------------------------------------
# ✅ Kick off the process with user input
# --------------------------------------
# Ask the human for a question
user_input = input("👤 Ask something: ")

initial_state = {
    "messages": [HumanMessage(content=user_input)],
    "approved": False,
    "retry_count": 0,      # ← Needed for loop control if you're using retry limits
    "rejections": []       # ← Needed if you're storing rejected attempts
}

👤 Ask something:  Whats up?


In [134]:
# --------------------------------------
# ✅ Run the workflow (stream = step-by-step execution)
# --------------------------------------
for step in workflow.stream(initial_state):
    for key, value in step.items():
        if key == "messages":
            #Print all AI Messages returned at this step
            for msg in value:
                print(msg.content)


 LLM Suggested:
I’m here to help if you have any questions or need assistance! How can I assist you today?


Approve this response? (y/n):  y


NameError: name 'approved' is not defined